> Run this cell once!

> This will tell python to reload before each import.

 >This enables you to see updated version of your module as you change and save.

In [ ]:
%load_ext autoreload
%autoreload 2

# External Modules

In [ ]:
# Code Here
import string # For string letters
import random # For randomly selection from lists
from fuzzywuzzy import fuzz # For string similarity comparison

# `Agent` Class

In [ ]:
# *------------------* #
# Agent Class
class Agent:
    """This class implements the 'Agent' for the evolution purpose.
    """
    # Contructor
    def __init__( self, string_length, initial_string = None ):
        # Store String Length
        self.string_length = string_length
        # If there is no string to initialize, randomly generate one
        if( initial_string is None ):
            self.string_content = ''.join( [ random.choice( string.ascii_letters + ' ' ) for _ in range( self.string_length ) ] )
        else:
            self.string_content = initial_string
        # Where fitness should be stored
        self.fitness = None
    # String
    def __str__( self ):
        string_representation = """String Length:  {}
String Content: {}
String Fitness: {}""".format( self.string_length, self.string_content, self.fitness )
        return( string_representation )

# `ga`: Main Genetic Algorithm Function

In [ ]:
# *------------------* #
# Do GA
def ga( N_population, N_generations, target_string ):
    """Do Genetic Algorithm Search
    """
    # Parameters
    string_length = len( target_string )
    # Report Format
    report_format = '--------------\nGeneration: {}\nBest Agent Results\n{}'
    # Initialization
    agents = initialization( N_population, string_length )
    ## Main Loop
    for i in range(N_generations):
        # Assign Fitness
        agents = fitness( agents, target_string  )
        # Selection
        agents = selection( agents, selection_ratio = 0.2 )
        # Early-Stopping Criteria
        best_fitness = agents[0].fitness
        if( best_fitness > 91 ):
            break
        #--------------------
        # Report
        if( (1 > 0) and ( i%10**3 == 0 ) ):
            print(report_format.format( i, str(agents[0]) )) # If you have implemented `__str__` for 'Agent'
        #--------------------
        # Cross-Over
        agents = crossover( agents, N_population )
        # Mutation
        agents = mutation( agents, selection_ratio = 0.2, hard_probability = 0.05, soft_probability = 0.25  )
    # Last Sort
    agents = fitness( agents, target_string )
    agents = sorted( agents, key = lambda agent: agent.fitness, reverse = True )
    # Report
    if( 1 > 0 ):
        print(report_format.format( i+1, str(agents[0]) ))
    # Retun
    return( agents )

# `initialization`: Initial Population

In [ ]:
# *------------------* #
# Initialize
def initialization( N_population, string_length ):
    """Initialize Agents
    """
    # Create and Initialize Agents
    agents = [ Agent( string_length ) for _ in range( N_population ) ]
    # Return
    return( agents )

# `fitness`: Calculating Fitness

In [ ]:
# *------------------* #
# Fitness
def fitness( agents, target_string ):
    """Calculate fitness for each agent.
    """
    # Calculate and Set Fitness of Agents
    for agent in agents:
        agent.fitness = fuzz.ratio( target_string, agent.string_content )
    # Return
    return( agents )

# `selection`: Survival of the Fittest

In [ ]:
# *------------------* #
# Selection
def selection( agents, selection_ratio ):
    """Select (a fraction) of fittest agents.
    """
    N = int( len( agents ) * selection_ratio )
    #####################
    # Find Fitness
    agents = sorted( agents, key=( lambda x: x.fitness ), reverse=True )
    agents = agents[ :N ] # Just Fittest!
#     agents = agents[ :int(N/2) ] + agents[ N:int(3*N/2) ] # Some Fittest and some further away
#     agents = agents[ :int(N/2) ] + random.sample( agents[ int(N/2): ], int(N/2) ) # Some Fittest and some random
    # Return
    return( agents )

# `crossover`: Breeding

In [ ]:
# *------------------* #
# Cross-Over
def crossover( agents, N_population ):
    """Cross over the agents.
    """
    # New Children!
    new_agents = []
    while( (len(new_agents) + len(agents)) < N_population ) : # While number of children is not too many
        # Parent
        parent1, parent2 = random.sample( agents, 2 )
        # Random Slice
        string_length = parent1.string_length
        indices = random.sample( list(range(string_length)), int(string_length/2) )
        # Cross Over
        child1_string = list(parent1.string_content)
        child2_string = list(parent2.string_content)
        # For Cross-Overed Indices
        for idx in indices:
            child1_string[idx] = parent2.string_content[idx]
            child2_string[idx] = parent1.string_content[idx]
        # Convert to string
        child1_string = ''.join( child1_string )
        child2_string = ''.join( child2_string )
        # Create children with given genes
        child1, child2 = Agent( string_length, initial_string=child1_string ), Agent( string_length, initial_string=child2_string )
        # Append
        new_agents += [ child1, child2 ]
    # Return
    return( agents + new_agents )

# `mutation`: New Features

In [ ]:
# *------------------* #
# Mutation
def mutation( agents, selection_ratio, hard_probability = 0.05, soft_probability = 0.25 ):
    """Mutate agents.
    """
    # Number to not mutate
    N = int( len(agents) * selection_ratio )
    # Loop through new children
    for agent in agents[ N: ]:
        u = random.random()
        if( u < hard_probability ): # Hard Mutation: Change all structure
            agent.string_content = ''.join( [ random.choice( string.ascii_letters + ' ' ) for _ in range(agent.string_length) ] )
        elif( u < (hard_probability + soft_probability) ): # Soft Mutation: Change one character
            idx = random.randint( 0, agent.string_length - 1 )
            agent.string_content = agent.string_content[:idx] + random.choice( string.ascii_letters+' ' ) + agent.string_content[(idx+1):]
    # Return
    return( agents )

# Putting it all together

In [ ]:
# *------------------------------* #
# Main
# Parameters
## Target
target_string = "Hello World"
## Simulation
N_population = 50
N_generations = 10**5
# Genetic Algorithm
agents = ga( N_population, N_generations, target_string )